<a href="https://colab.research.google.com/github/fajemila/taikwinmu/blob/main/fr2ewetranslation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.1.1

     |████████████████████████████████| 1.5MB 9.3MB/s 
     |████████████████████████████████| 890kB 34.9MB/s 
     |████████████████████████████████| 2.9MB 39.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=1ff48c97612b92f22a9de2a023ff8c75fc55c90d48ce83cc6d96fd7f16d52da1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
!pip install torchtext==0.6.0

     |████████████████████████████████| 71kB 5.8MB/s 
     |████████████████████████████████| 1.2MB 11.5MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [13]:
!python -m spacy download fr_core_news_sm

     |████████████████████████████████| 14.7MB 842kB/s 
  Created wheel for fr-core-news-sm: filename=fr_core_news_sm-2.2.5-cp36-none-any.whl size=14727027 sha256=ca55db79f521ad149e1d9c85cd92330ae844d4294408cee26d4492bb332e7b6a
  Stored in directory: /tmp/pip-ephem-wheel-cache-c_jhi4kq/wheels/46/1b/e6/29b020e3f9420a24c3f463343afe5136aaaf955dbc9e46dfc5
Successfully built fr-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
from transformers import AdamW

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.data import Field, BucketIterator
import string
import spacy
from torchtext import data

import numpy as np

import pandas as pd
import random
import math
import time

In [4]:
import torchtext
torchtext.__version__

'0.6.0'

In [5]:
train = pd.read_csv('drive/MyDrive/takwinmu/Train.csv')
test = pd.read_csv('drive/MyDrive/takwinmu/Test.csv')

In [6]:
train.head()

,ID,French,Target_Language,Target
0,ID_AADNDxdl,Mon père,Fon,Tɔ ce
1,ID_AAFQhmDr,Mettez-vous en rang.,Fon,Mi tò miɖéé
2,ID_AAHVDMdq,"Sénégal, Côte d'Ivoire, Guinée, Ghana, on déco...",Ewe,"Sénégal, Côte d'Ivoire, Guinée, Ghana, siwo ƒe..."
3,ID_AAJfVHEH,Son doigt lui fait mal.,Fon,Alɔvi tɔn ɖo vivɛ wɛ
4,ID_AAOJuhzN,La pluie a commencé.,Fon,Jì bɛ́


In [7]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [8]:
spacy_fr = spacy.load('fr_core_news_sm')
spacy_en = spacy.load('en')

In [9]:
ewedf = train[train['Target_Language']=="Ewe"]
ewedftest = test[test['Target_Language']=="Ewe"]
ewedf.to_csv('ewedf.csv',index=False)
ewedftest.to_csv('ewedftest.csv',index=False)

In [10]:
ewedf.head()

,ID,French,Target_Language,Target
2,ID_AAHVDMdq,"Sénégal, Côte d'Ivoire, Guinée, Ghana, on déco...",Ewe,"Sénégal, Côte d'Ivoire, Guinée, Ghana, siwo ƒe..."
6,ID_AARXSjjg,Janot se prit à grelotter dès que le soleil se...,Ewe,Yano dze ƒoƒo esi me ɣe gbe ɖo to eye ya dze ƒ...
13,ID_AAmSrrNh,"Et cela en une journée, sinon rien à manger.",Ewe,"Nawɔe le ŋkekea me. Nemenyυo oa, atdi adɔ"
17,ID_AAsKjVJM,L’idée est partie de deux incidents survenus l...,Ewe,"nua dze eg,ome tso masɔmasɔ aɖe siwo do mo ɖa ..."
19,ID_ABAUPxlf,Mais je souris quand même parce que ça fait pa...,Ewe,kehã meko nu elabe esia hã le dɔa wɔwɔ me


In [11]:
def tokenize_fr(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_fr.tokenizer(text)]
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [12]:
ID = Field()
SRC = Field(tokenize = tokenize_fr, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True,
            batch_first = True,
            fix_length=50)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True,
            batch_first = True,
            fix_length=50)

In [13]:
from torchtext import data
tv_datafields = [('ID', None),("French", SRC),('Lang',None), ("Target", TRG)]

ewetrain_data = data.TabularDataset(path='ewedf.csv',format="csv",
                                                skip_header=True, fields =tv_datafields)
ewetest_data = data.TabularDataset(path='ewedftest.csv', format="csv",
                                                skip_header=True, fields =[('ID', None),("French", SRC)])

In [14]:
print(f"Number of training examples: {len(ewetrain_data.examples)}")
print(f"Number of testing examples: {len(ewetest_data.examples)}")

Number of training examples: 22353
Number of testing examples: 2964


In [15]:
print(vars(ewetrain_data.examples[0]))

{'French': ['sénégal', ',', 'côte', "d'", 'ivoire', ',', 'guinée', ',', 'ghana', ',', 'on', 'découvre', 'une', 'afrique', 'dont', 'les', 'réalités', 'diffèrent', "d'", 'un', 'pays', 'à', '\xa0 ', 'un', 'autre', 'mais', 'qui', 'partout', 'garde', 'des', 'traces', 'de', 'la', 'récente', 'colonisation', ','], 'Target': ['sénégal', ',', 'côte', "d'ivoire", ',', 'guinée', ',', 'ghana', ',', 'siwo', 'ƒe', 'nuwo', 'to', 'vovo', 'gake', 'woƒe', 'amediɖuɖu', 'teƒewo', 'mebu', 'o']}


In [16]:
import random

ewetrain_data, ewevalid_data = ewetrain_data.split(split_ratio=0.9,random_state = random.seed(SEED))

In [17]:
SRC.build_vocab(ewetrain_data, min_freq = 2)
TRG.build_vocab(ewetrain_data, min_freq = 2)

In [18]:
print(f"Unique tokens in source (yr) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (yr) vocabulary: 13264
Unique tokens in target (en) vocabulary: 9581


In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [20]:
ewetrain_iter , ewevalid_iter = data.BucketIterator.splits((ewetrain_data,ewevalid_data), device=device, 
                                 batch_sizes=(16,16),
                                 sort_key = lambda x: data.interleave_keys(len(x.French),len(x.Target)),
                                 sort_within_batch=True, repeat=False)
ewetest_iter = data.Iterator(ewetest_data,batch_size=16,device=device,train=False,sort=False,sort_within_batch=False)

In [21]:
class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim,
                 dropout, 
                 device,
                 max_length = 50):
        super().__init__()

        self.device = device
        
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim,
                                                  dropout, 
                                                  device) 
                                     for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len]
        #src_mask = [batch size, src len]
        
        batch_size = src.shape[0]
        src_len = src.shape[1]
        
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #pos = [batch size, src len]
        
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        
        #src = [batch size, src len, hid dim]
        
        for layer in self.layers:
            src = layer(src, src_mask)
            
        #src = [batch size, src len, hid dim]
            
        return src

In [22]:
class EncoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim,  
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len, hid dim]
        #src_mask = [batch size, src len]
                
        #self attention
        _src, _ = self.self_attention(src, src, src, src_mask)
        
        #dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        #positionwise feedforward
        _src = self.positionwise_feedforward(src)
        
        #dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        return src

In [23]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask = None):
        
        batch_size = query.shape[0]
        
        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]
                
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
        
        #Q = [batch size, query len, hid dim]
        #K = [batch size, key len, hid dim]
        #V = [batch size, value len, hid dim]
                
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        
        #Q = [batch size, n heads, query len, head dim]
        #K = [batch size, n heads, key len, head dim]
        #V = [batch size, n heads, value len, head dim]
                
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        
        #energy = [batch size, n heads, query len, key len]
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        
        attention = torch.softmax(energy, dim = -1)
                
        #attention = [batch size, n heads, query len, key len]
                
        x = torch.matmul(self.dropout(attention), V)
        
        #x = [batch size, n heads, query len, head dim]
        
        x = x.permute(0, 2, 1, 3).contiguous()
        
        #x = [batch size, query len, n heads, head dim]
        
        x = x.view(batch_size, -1, self.hid_dim)
        
        #x = [batch size, query len, hid dim]
        
        x = self.fc_o(x)
        
        #x = [batch size, query len, hid dim]
        
        return x, attention

In [24]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [batch size, seq len, hid dim]
        
        x = self.dropout(torch.relu(self.fc_1(x)))
        
        #x = [batch size, seq len, pf dim]
        
        x = self.fc_2(x)
        
        #x = [batch size, seq len, hid dim]
        
        return x

In [25]:
class Decoder(nn.Module):
    def __init__(self, 
                 output_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device,
                 max_length = 50):
        super().__init__()
        
        self.device = device
        
        self.tok_embedding = nn.Embedding(output_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([DecoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim, 
                                                  dropout, 
                                                  device)
                                     for _ in range(n_layers)])
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, trg len]
        #src_mask = [batch size, src len]
                
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
                            
        #pos = [batch size, trg len]
            
        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))
                
        #trg = [batch size, trg len, hid dim]
        
        for layer in self.layers:
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        output = self.fc_out(trg)
        
        #output = [batch size, trg len, output dim]
            
        return output, attention

In [26]:
class DecoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len, hid dim]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, trg len]
        #src_mask = [batch size, src len]
        
        #self attention
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)
        
        #dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
            
        #trg = [batch size, trg len, hid dim]
            
        #encoder attention
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)
        
        #dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
                    
        #trg = [batch size, trg len, hid dim]
        
        #positionwise feedforward
        _trg = self.positionwise_feedforward(trg)
        
        #dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return trg, attention

In [27]:
class Seq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 src_pad_idx, 
                 trg_pad_idx, 
                 device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
    def make_src_mask(self, src):
        
        #src = [batch size, src len]
        
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        #src_mask = [batch size, 1, 1, src len]

        return src_mask
    
    def make_trg_mask(self, trg):
        
        #trg = [batch size, trg len]
        
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        
        #trg_pad_mask = [batch size, 1, 1, trg len]
        
        trg_len = trg.shape[1]
        
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()
        
        #trg_sub_mask = [trg len, trg len]
            
        trg_mask = trg_pad_mask & trg_sub_mask
        
        #trg_mask = [batch size, 1, trg len, trg len]
        
        return trg_mask

    def forward(self, src, trg):
        
        #src = [batch size, src len]
        #trg = [batch size, trg len]
                
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        
        #src_mask = [batch size, 1, 1, src len]
        #trg_mask = [batch size, 1, trg len, trg len]
        
        enc_src = self.encoder(src, src_mask)
        
        #enc_src = [batch size, src len, hid dim]
                
        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)
        
        #output = [batch size, trg len, output dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return output, attention

In [28]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
HID_DIM = 256
ENC_LAYERS = 2
DEC_LAYERS = 1
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)

dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device)

In [29]:
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [30]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 10,181,229 trainable parameters


In [31]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

In [32]:
model.apply(initialize_weights);

In [33]:
LEARNING_RATE = 0.0005

optimizer = AdamW(model.parameters(), lr = LEARNING_RATE)
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [34]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.French
        trg = batch.Target
        
        optimizer.zero_grad()
        
        output, _ = model(src, trg[:,:-1])
        
        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
        
        output_dim = output.shape[-1]
        
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
        
        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [35]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.French
            trg = batch.Target

            output, _ = model(src, trg[:,:-1])
        
            #output = [batch size, trg len - 1, output dim]
            #trg = [batch size, trg len]

            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)

            #output = [batch size * trg len - 1, output dim]
            #trg = [batch size * trg len - 1]
            
            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [36]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [37]:
N_EPOCHS = 10
CLIP = 1
from torch.nn import functional as F
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, ewetrain_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, ewevalid_iter, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 57s
	Train Loss: 5.639 | Train PPL: 281.242
	 Val. Loss: 4.971 |  Val. PPL: 144.109
Epoch: 02 | Time: 0m 56s
	Train Loss: 4.685 | Train PPL: 108.341
	 Val. Loss: 4.664 |  Val. PPL: 106.022
Epoch: 03 | Time: 0m 57s
	Train Loss: 4.095 | Train PPL:  60.042
	 Val. Loss: 4.614 |  Val. PPL: 100.870
Epoch: 04 | Time: 0m 56s
	Train Loss: 3.585 | Train PPL:  36.069
	 Val. Loss: 4.615 |  Val. PPL: 100.992
Epoch: 05 | Time: 0m 56s
	Train Loss: 3.124 | Train PPL:  22.748
	 Val. Loss: 4.729 |  Val. PPL: 113.201
Epoch: 06 | Time: 0m 56s
	Train Loss: 2.716 | Train PPL:  15.121
	 Val. Loss: 4.886 |  Val. PPL: 132.445
Epoch: 07 | Time: 0m 56s
	Train Loss: 2.368 | Train PPL:  10.680
	 Val. Loss: 5.078 |  Val. PPL: 160.459
Epoch: 08 | Time: 0m 56s
	Train Loss: 2.080 | Train PPL:   8.003
	 Val. Loss: 5.286 |  Val. PPL: 197.510
Epoch: 09 | Time: 0m 57s
	Train Loss: 1.842 | Train PPL:   6.310
	 Val. Loss: 5.480 |  Val. PPL: 239.799
Epoch: 10 | Time: 0m 56s
	Train Loss: 1.655 | Train PPL

In [38]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, ewevalid_iter, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 4.614 | Test PPL: 100.870 |


In [39]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):
    
    model.eval()

    if isinstance(sentence, str):
        nlp = spacy.load('fr')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    
    src_mask = model.make_src_mask(src_tensor)
    
    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        trg_mask = model.make_trg_mask(trg_tensor)
        
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:], attention

In [40]:
example_idx = 5

src = vars(ewetrain_data.examples[example_idx])['French']
trg = vars(ewetrain_data.examples[example_idx])['Target']

print(f'French = {src}')
print(f'Target = {trg}')

French = ['je', 'vous', 'recommande', 'définitivement', 'smartcup', 'si', 'vous', 'avez', 'besoin', 'de', 'quelqu’', 'un', 'pour', 'gérer', 'vos', 'cocktails', 'à', 'un', 'évènement', ',', 'et', '\xa0', 'your', 'amazing', 'party', '\xa0', 'de', 'frédérique', 'et', 'laurie', 'pour', 's’', 'assurer', 'que', 'tout', 'se', 'passe', 'bien', ',', 'de', 'la', 'décoration', 'à', 'la', 'nourriture']
Target = ['medo', 'ŋusẽ', 'mi', 'be', 'miayɔ', 'smartcup', 'kple', 'frederique', 'kple', 'lorie', 'ƒe', '"', 'your', 'amazing', 'party', "'", 'ne', 'woakpɔ', 'miaƒe', 'azãɖuɖu', 'na', 'mi']


In [41]:
translation, attention = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted English = {translation}')

predicted English = ['medi', 'be', 'mayi', 'ɖa', 'ƒle', '<unk>', 'eye', '<unk>', '<unk>', '<unk>', '<unk>', '<eos>']


In [42]:
trgs = []
pred_trgs = []

for datum in ewetest_data:
    
    src = vars(datum)['French']
    if len(src)>48:
      src= src[:48]
    pred_trg, _ = translate_sentence(src, SRC, TRG, model, device, 50)
    
    #cut off <eos> token
    pred_trg = pred_trg[:-1]
    
    pred_trgs.append(pred_trg)    

In [43]:
ewedftest.head()

,ID,French,Target_Language
0,ID_AAAAhgRX,Très fière d’elle,Ewe
4,ID_AChdWHyF,Grosse bagnolle,Ewe
11,ID_AHBSoUNL,Les seins comme ça… » Basta,Ewe
14,ID_AHycIkQv,Lire aussi Pensées Nocturnes,Ewe
16,ID_AIWTdKBT,"voir même de la positivité, de la gaieté et po...",Ewe


In [44]:
ewedftest['Target'] = pred_trgs

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [45]:
ewedftest.head()

,ID,French,Target_Language,Target
0,ID_AAAAhgRX,Très fière d’elle,Ewe,"[ewɔ, nuku, nam, be, <unk>]"
4,ID_AChdWHyF,Grosse bagnolle,Ewe,"[<unk>, <unk>]"
11,ID_AHBSoUNL,Les seins comme ça… » Basta,Ewe,"[<unk>, nu, sia, o]"
14,ID_AHycIkQv,Lire aussi Pensées Nocturnes,Ewe,"[ke, hã, <unk>]"
16,ID_AIWTdKBT,"voir même de la positivité, de la gaieté et po...",Ewe,"[ke, hã, nye, <unk>, le, nye, <unk>, me, eye,..."


In [46]:
ewedftest['Target'] = [' '.join(map(str, l)) for l in ewedftest['Target']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [47]:
ewedftest.head()

,ID,French,Target_Language,Target
0,ID_AAAAhgRX,Très fière d’elle,Ewe,ewɔ nuku nam be <unk>
4,ID_AChdWHyF,Grosse bagnolle,Ewe,<unk> <unk>
11,ID_AHBSoUNL,Les seins comme ça… » Basta,Ewe,<unk> nu sia o
14,ID_AHycIkQv,Lire aussi Pensées Nocturnes,Ewe,ke hã <unk>
16,ID_AIWTdKBT,"voir même de la positivité, de la gaieté et po...",Ewe,ke hã nye <unk> le nye <unk> me eye <unk> nu o


In [48]:
ewedftest.to_csv('ewesol.csv',index=False)